<a href="https://colab.research.google.com/github/amarnath-a-ai/customer-support-ticket-triage/blob/main/Week1_Customer_Support_Triage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from datetime import timedelta


file_path = "/customer_support_tickets.csv"
df = pd.read_csv(file_path)

print("=== DATASET OVERVIEW ===")
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])
print("Column names:", df.columns.tolist())

text_keywords = ["message", "description", "issue", "ticket"]
time_keywords = ["created", "date", "time", "timestamp"]

text_col = next(
    (c for c in df.columns if any(k in c.lower() for k in text_keywords)),
    None
)

time_col = next(
    (c for c in df.columns if any(k in c.lower() for k in time_keywords)),
    None
)

print("Detected text column:", text_col)
print("Detected time column:", time_col)

if text_col is None or time_col is None:
    raise ValueError("Required text or time column not found")

df[time_col] = pd.to_datetime(df[time_col], errors="coerce")


df["clean_message"] = (
    df[text_col]
    .astype(str)
    .str.lower()
    .str.replace(r"[^a-z0-9\s]", " ", regex=True)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)


def classify_issue(msg):
    if any(k in msg for k in ["payment", "card", "upi"]):
        return "PAYMENT"
    if any(k in msg for k in ["login", "password", "otp"]):
        return "LOGIN"
    if any(k in msg for k in ["delivery", "shipping", "courier"]):
        return "DELIVERY"
    if any(k in msg for k in ["refund", "return"]):
        return "REFUND"
    if any(k in msg for k in ["bug", "error", "crash"]):
        return "BUG"
    return "GENERAL"

df["issue_type"] = df["clean_message"].apply(classify_issue)

def assign_priority(msg):
    if any(k in msg for k in ["urgent", "system down", "blocked"]):
        return "P0"
    if any(k in msg for k in ["failed", "error", "not working"]):
        return "P1"
    if any(k in msg for k in ["delay", "slow"]):
        return "P2"
    return "P3"

df["priority"] = df["clean_message"].apply(assign_priority)

sla_map = {"P0": 2, "P1": 6, "P2": 24, "P3": 72}
df["sla_hours"] = df["priority"].map(sla_map)
df["due_time"] = df[time_col] + pd.to_timedelta(df["sla_hours"], unit="h")

print("\n=== SUPPORT MANAGER REPORT ===")
print("Total Tickets:", len(df))
print("\nBy Issue Type:\n", df["issue_type"].value_counts())
print("\nBy Priority:\n", df["priority"].value_counts())
print("\nOverdue Tickets:", (df["due_time"] < pd.Timestamp.now()).sum())


output_path = "/final_customer_support_output.csv"
df.to_csv(output_path, index=False)

print("\nSUCCESS: Processed file saved to:", output_path)

=== DATASET OVERVIEW ===
Rows: 8469
Columns: 17
Column names: ['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age', 'Customer Gender', 'Product Purchased', 'Date of Purchase', 'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status', 'Resolution', 'Ticket Priority', 'Ticket Channel', 'First Response Time', 'Time to Resolution', 'Customer Satisfaction Rating']
Detected text column: Ticket ID
Detected time column: Date of Purchase

=== SUPPORT MANAGER REPORT ===
Total Tickets: 8469

By Issue Type:
 issue_type
GENERAL    8469
Name: count, dtype: int64

By Priority:
 priority
P3    8469
Name: count, dtype: int64

Overdue Tickets: 8469

SUCCESS: Processed file saved to: /final_customer_support_output.csv
